In [ ]:
#dependencies
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Re

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tokenizing").getOrCreate()

In [ ]:
#read csv file
#smsbase_df=pd.read_csv("Resources/sms_spam.csv", encoding = 'ISO-8859-1', usecols=[0,1])
#smsbase_df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from pyspark import SparkFiles
#url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_1/nutrition.csv"
#smsbase_df=pd.read_csv("Resources/sms_spam.csv", encoding = 'ISO-8859-1', usecols=[0,1])
spark.sparkContext.addFile("Resources/sms_spam.csv")
smsbase_df = spark.read.csv(SparkFiles.get("sms_spam.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True) #Observe the need to use ignoreLeadingWhiteSpace=True, otherwise a leading whitespace will appear in the column names

# Show DataFrame
smsbase_df.show() 

+----+--------------------+----+----+----+
|  v1|                  v2| _c2| _c3| _c4|
+----+--------------------+----+----+----+
| ham|Go until jurong p...|null|null|null|
| ham|Ok lar... Joking ...|null|null|null|
|spam|Free entry in 2 a...|null|null|null|
| ham|U dun say so earl...|null|null|null|
| ham|Nah I don't think...|null|null|null|
|spam|FreeMsg Hey there...|null|null|null|
| ham|Even my brother i...|null|null|null|
| ham|As per your reque...|null|null|null|
|spam|WINNER!! As a val...|null|null|null|
|spam|Had your mobile 1...|null|null|null|
| ham|I'm gonna be home...|null|null|null|
|spam|SIX chances to wi...|null|null|null|
|spam|URGENT! You have ...|null|null|null|
| ham|I've been searchi...|null|null|null|
| ham|I HAVE A DATE ON ...|null|null|null|
|spam|XXXMobileMovieClu...|null|null|null|
| ham|Oh k...i'm watchi...|null|null|null|
| ham|Eh u remember how...|null|null|null|
| ham|Fine if that��s t...|null|null|null|
|spam|England v Macedon...|null|null|null|
+----+-----

In [ ]:
#columns = ["v1", "v2"]
sms_df = smsbase_df.select(("v1"), ("v2"))

sms_df.show()

+----+--------------------+
|  v1|                  v2|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if that��s t...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [ ]:
sms_df.dropDuplicates(subset=['v2'])
print(sms_df)

DataFrame[v1: string, v2: string]


In [ ]:
sms_df.sort("v1").show()

+---+--------------------+
| v1|                  v2|
+---+--------------------+
|ham|I���m going to tr...|
|ham| WHO ARE YOU SEEING?|
|ham|So �_ pay first l...|
|ham|Ok lar... Joking ...|
|ham|Aft i finish my l...|
|ham|U dun say so earl...|
|ham|Ffffffffff. Alrig...|
|ham|Did you catch the...|
|ham|I'm back &amp; we...|
|ham|As per your reque...|
|ham|Ahhh. Work. I vag...|
|ham|I've been searchi...|
|ham|Wait that's still...|
|ham|Oh k...i'm watchi...|
|ham|Yeah he got in at...|
|ham|Fine if that��s t...|
|ham|K tell me anythin...|
|ham|Go until jurong p...|
|ham|For fear of faint...|
|ham|Even my brother i...|
+---+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:

tokened = Tokenizer(inputCol="v2", outputCol="words")
tokenized = tokened.transform(sms_df)
tokenized.show()
spark = SparkSession.builder.appName("tokenizing").getOrCreate()

+----+--------------------+--------------------+
|  v1|                  v2|               words|
+----+--------------------+--------------------+
| ham|Go until jurong p...|[go, until, juron...|
| ham|Ok lar... Joking ...|[ok, lar..., joki...|
|spam|Free entry in 2 a...|[free, entry, in,...|
| ham|U dun say so earl...|[u, dun, say, so,...|
| ham|Nah I don't think...|[nah, i, don't, t...|
|spam|FreeMsg Hey there...|[freemsg, hey, th...|
| ham|Even my brother i...|[even, my, brothe...|
| ham|As per your reque...|[as, per, your, r...|
|spam|WINNER!! As a val...|[winner!!, as, a,...|
|spam|Had your mobile 1...|[had, your, mobil...|
| ham|I'm gonna be home...|[i'm, gonna, be, ...|
|spam|SIX chances to wi...|[six, chances, to...|
|spam|URGENT! You have ...|[urgent!, you, ha...|
| ham|I've been searchi...|[i've, been, sear...|
| ham|I HAVE A DATE ON ...|[i, have, a, date...|
|spam|XXXMobileMovieClu...|[xxxmobilemoviecl...|
| ham|Oh k...i'm watchi...|[oh, k...i'm, wat...|
| ham|Eh u remember 

In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [ ]:
# Create a user defined function 
count_tokens = udf(word_list_length, IntegerType())
count_tokens


<function __main__.word_list_length(word_list)>

In [ ]:
tokens_df = tokenized.select("v1", "words")\
    .withColumn("tokens", count_tokens(col("words"))) 

In [ ]:
tokens_df = tokenized.select("v1", "words").withColumn("tokens", col("words")) 
  


In [ ]:
tokens_df

DataFrame[v1: string, words: array<string>, tokens: int]

In [ ]:
tokens_df.show(truncate=False)

Py4JJavaError: ignored

In [ ]:
# Split our preprocessed data into our features and target arrays
X=spambase_df.drop("class",axis=1).values
y=spambase_df["class"].values

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
     

In [ ]:
#create a standardscaler instances
scaler=StandardScaler()
#fit the standard scaler
X_scaler=scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
     

Compile, Train and Evaluate the Model


In [ ]:
# Get the input feature/ shape
X_train_scaled.shape
     

(3450, 57)

Neural network optimization


In [ ]:
!pip install keras-tuner
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.6 MB/s eta 0:00:00


In [ ]:
nn_model1 = tf.keras.models.Sequential()
input_features = X_train_scaled.shape[1]

# First hidden layer
nn_model1.add(tf.keras.layers.Dense(units=10, activation='relu', input_dim=input_features))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer

nn_model1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn_model1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                580       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 701
Trainable params: 701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model

fit_model_1 = nn_model1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
108/108 [==============================] - 3s 6ms/step - loss: 0.5984 - accuracy: 0.6675
Epoch 2/50
108/108 [==============================] - 0s 4ms/step - loss: 0.3928 - accuracy: 0.8278
Epoch 3/50
108/108 [==============================] - 0s 4ms/step - loss: 0.2919 - accuracy: 0.8965
Epoch 4/50
108/108 [==============================] - 0s 4ms/step - loss: 0.2470 - accuracy: 0.9125
Epoch 5/50
108/108 [==============================] - 0s 3ms/step - loss: 0.2222 - accuracy: 0.9191
Epoch 6/50
108/108 [==============================] - 0s 3ms/step - loss: 0.2055 - accuracy: 0.9272
Epoch 7/50
108/108 [==============================] - 0s 3ms/step - loss: 0.1936 - accuracy: 0.9328
Epoch 8/50
108/108 [==============================] - 0s 2ms/step - loss: 0.1843 - accuracy: 0.9348
Epoch 9/50
108/108 [==============================] - 0s 3ms/step - loss: 0.1772 - accuracy: 0.9371
Epoch 10/50
108/108 [==============================] - 0s 2ms/step - loss: 0.1710 - accuracy: 0.9386

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

36/36 - 0s - loss: 0.1654 - accuracy: 0.9479 - 427ms/epoch - 12ms/step
Loss: 0.16539061069488525, Accuracy: 0.9478713870048523


In [ ]:
# Export our model to HDF5 file
nn_model1.save('../h5_files/spamcsv_model_1.h5')